# ***COVID-19: Understanding the Growth Factor involved in the exponential growth of the Corona Virus & Inflection Point in the Curve of Growth***

![](https://miro.medium.com/max/4800/0*2Hb-dGkPAZU_vUtW)

Photo by Viktor Forgacs on Unsplash

#### For all that's happening with the Coronavirus, deciding on a decision about what to do today can be incredibly hard.

### Will it be a good
### idea to wait for more details?
### Will anything work out today?
### The best way to avoid this is to reduce the growth factor in the number of instances correlated with an exponential increase. But what does that factor for growth consist of? 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import folium
from scipy.integrate import odeint
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df1 = pd.read_csv('../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv')
df1.head(2)

In [ ]:
df1 = df1[df1.columns[:-8]]
df1.drop('Unnamed: 3', axis=1, inplace=True)

In [ ]:
df1.head(2)

In [ ]:
df1['reporting date'] = pd.to_datetime(df1['reporting date'])
df1['exposure_start'] = pd.to_datetime(df1['exposure_start'])
df1['exposure_end'] = pd.to_datetime(df1['exposure_end'])
df1['hosp_visit_date'] = pd.to_datetime(df1['hosp_visit_date'])
df1['symptom_onset'] = pd.to_datetime(df1['symptom_onset'])

In [ ]:
df2 = pd.read_csv('../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv')
df2 = df2[df2.columns[:-12]]
df2.loc[df2['sex']=='male', 'sex'] = 'Male'
df2.loc[df2['sex']=='female', 'sex'] = 'Female'
df2.head(2)

In [ ]:
df3 = pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv")
df3.head()

### **Total countries and locations affected**

In [ ]:
df_loc = pd.DataFrame(df1.groupby(['country'])['location'].nunique()).reset_index().sort_values(by='location', ascending=False).reset_index(drop=True)
df_loc.loc[df_loc.shape[0]]=['Total: '+str(df_loc['country'].nunique()), 'Total: '+str(df_loc['location'].sum())]
df_loc

In [ ]:
df1.head(2)

In [ ]:
df1['sym_exp_diff'] = (df1['symptom_onset'] - df1['exposure_end']).dt.days
df1['hosp_sym_diff'] = (df1['hosp_visit_date'] - df1['symptom_onset']).dt.days

In [ ]:
fig = px.pie(df1, values=[df1['gender'].value_counts()[0], df1['gender'].value_counts()[1]], names=['Male', 'Female'], title='Male v Female Affected Ratio')
fig.show()

In [ ]:
fig = px.violin(df2[df2['sex']!='4000'].dropna(subset=['age', 'sex']), y="age", x='sex', color="sex",
                hover_data=df2.columns, title='Age Ratio of people affected b/w the two genders')
fig.show()

- Comparing both of them seems like the age of the affected females are higher than that of males

##### Plotting the locations on the map

In [ ]:
# Credits: https://www.kaggle.com/imdevskp/covid-19-analysis-visualization-comparisons, Devakumar Kp

m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

df3_mapping = df3.dropna(subset=['Confirmed']).reset_index(drop=True)

for i in range(0, len(df3_mapping)):
    folium.Circle(
        location=[df3_mapping.iloc[i]['Lat'], df3_mapping.iloc[i]['Long']],
        color='crimson', 
        tooltip =   '<li><bold>Country : '+str(df3_mapping.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(df3_mapping.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(df3_mapping.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(df3_mapping.iloc[i]['Deaths'])+
                    '<li><bold>Recovered : '+str(df3_mapping.iloc[i]['Recovered']),
        radius=int(df3_mapping.iloc[i]['Confirmed'])**1.1).add_to(m)
m

In [ ]:
ncov_df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')

ncov_df['ObservationDate'] = pd.to_datetime(ncov_df['ObservationDate']) 

ncov_df["Country"] = ncov_df["Country/Region"].replace(
    {
        "Mainland China": "China",
        "Hong Kong SAR": "Hong Kong",
        "Taipei and environs": "Taiwan",
        "Iran (Islamic Republic of)": "Iran",
        "Republic of Korea": "South Korea",
        "Republic of Ireland": "Ireland",
        "Macao SAR": "Macau",
        "Russian Federation": "Russia",
        "Republic of Moldova": "Moldova",
        "Taiwan*": "Taiwan",
        "Cruise Ship": "Others",
        "United Kingdom": "UK",
        "Viet Nam": "Vietnam",
        "Czechia": "Czech Republic",
        "St. Martin": "Saint Martin",
        "Cote d'Ivoire": "Ivory Coast",
        "('St. Martin',)": "Saint Martin",
        "Congo (Kinshasa)": "Congo",
    }
)
ncov_df["Province"] = ncov_df["Province/State"].fillna("-").replace(
    {
        "Cruise Ship": "Diamond Princess cruise ship",
        "Diamond Princess": "Diamond Princess cruise ship"
    }
)

In [ ]:
ncov_df.head()

In [ ]:
ncov_china = ncov_df[ncov_df['Country'] == 'China']
ncov_china = pd.DataFrame(ncov_china.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=ncov_china['ObservationDate'], y=ncov_china['Confirmed'], name='Confirmed Cases'))
fig1.add_trace(go.Scatter(x=ncov_china[21:23]['ObservationDate'], y=ncov_china[21:23]['Confirmed'], mode='markers', name='Inflection', marker=dict(color='Red',line=dict(width=5, color='Red'))))
fig1.layout.update(title_text='COVID-19 Growth in China & Inflection',xaxis_showgrid=False, yaxis_showgrid=False, width=800,
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig1.layout.plot_bgcolor = 'Black'
fig1.layout.paper_bgcolor = 'Black'
fig1.show()

# # # From the above picture, I assume that the point of inflection is between the two red dots somewhere in the area. We note that as of around 1 March the linear curve starts going upwards. The growth factor for the next two days looks to be a steady ~1 #
# # So why did the curve go up so rapidly in the first place? What has made
the development so rapid? 


![](https://internetmonk.com/wp-content/uploads/Covid19ChinaExplained-2048x1150.png)

Source: Tomas Pueyo analysis over chart from the Journal of the American Medical Association, based on raw case data from the Chinese Center for Disease Control and Prevention

### In the graph above we see that the gray bars apply to people's real no. and the yellow bars correlate to the individuals who have been checked. That means the ones who weren't checked got contact with a lot of people around. 

In [ ]:
fig = px.box(df1.dropna(subset=['sym_exp_diff']), y="sym_exp_diff", points='all', title='Days difference between symptom and exposure dates') # typically after how many days after the exposure do the symptoms come up
# fig.layout.plot_bgcolor = '#6A7806'
# fig.layout.paper_bgcolor = '#6A7806'
fig.show()

### - We note that signs occurred > = 0 days after their last exposure date for most than the majority of people. 

### Yet what did they do as the signs came up? Did they report to doctor on time? 

In [ ]:
fig = px.box(df1.dropna(subset=['hosp_sym_diff']).reset_index(drop=True), y="hosp_sym_diff", points='all', \
             title='Days difference between hospital visit and symptom dates') # typically after how many days after the exposure do the symptoms come up
fig.show()

### - They note that just a few days after the day of their symptoms there are so many patients returning to the hospital. Could this delay cause greater visibility and thereby extend it to more people? 

# ***First set of Growth Factor Hypothesis:***
### ***i) It includes mainly public exposure***
### ***ii) Secondly, to some extent, it also includes a person's delay in getting tested from the date of symptom***
### ***iii} through unnoticed items such as your smartphones, etc. Smartphone may be a easily infected object ***

#### The second theory could be very strongly associated with the first 

#### Lets check the top 10 countries with maximum cases and respective deaths and recoveries

In [ ]:
ncov_italy = ncov_df[ncov_df['Country'] == 'Italy']
ncov_us = ncov_df[ncov_df['Country'] == 'US']
ncov_spain = ncov_df[ncov_df['Country'] == 'Spain']
ncov_germany = ncov_df[ncov_df['Country'] == 'Germany']
ncov_iran = ncov_df[ncov_df['Country'] == 'Iran']
ncov_france = ncov_df[ncov_df['Country'] == 'France']
ncov_uk = ncov_df[ncov_df['Country'] == 'UK']
ncov_swiss = ncov_df[ncov_df['Country'] == 'Switzerland']
ncov_soukor = ncov_df[ncov_df['Country'] == 'South Korea']

ncov_italy = pd.DataFrame(ncov_italy.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_us = pd.DataFrame(ncov_us.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_spain = pd.DataFrame(ncov_spain.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_germany = pd.DataFrame(ncov_germany.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_iran = pd.DataFrame(ncov_iran.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_france = pd.DataFrame(ncov_france.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_uk = pd.DataFrame(ncov_uk.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_swiss = pd.DataFrame(ncov_swiss.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()
ncov_soukor = pd.DataFrame(ncov_soukor.groupby(['ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index()

In [ ]:
ncov_all = pd.DataFrame(ncov_df.groupby(['Country', 'ObservationDate'])['Confirmed', 'Deaths', 'Recovered'].sum()).reset_index().drop_duplicates(subset=['Country'], keep='last')
ncov_all.reset_index(drop=True, inplace=True)
ncov_all = ncov_all.sort_values(by=['Confirmed'], ascending=False).reset_index(drop=True)
ncov_all = ncov_all.head(10)

In [ ]:
fig = go.Figure(data=[
    go.Bar(name='Confirmed',x=ncov_all['Country'].unique(), y=ncov_all['Confirmed']),
    go.Bar(name='Deaths', x=ncov_all['Country'].unique(), y=ncov_all['Deaths']),
    go.Bar(name='Recovered', x=ncov_all['Country'].unique(), y=ncov_all['Recovered'])
])
# Change the bar mode
fig.layout.update(barmode='stack', title='Top 10 Country-wise Corona Cases & Consequences', yaxis_showgrid=False)
fig.show()

### How Corona Virus performed in Rest of the World

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=ncov_italy['ObservationDate'], y=ncov_italy['Confirmed'], name='Italy'))
fig1.add_trace(go.Scatter(x=ncov_us['ObservationDate'], y=ncov_us['Confirmed'], name='USA'))
fig1.add_trace(go.Scatter(x=ncov_spain['ObservationDate'], y=ncov_spain['Confirmed'], name='Spain'))
fig1.add_trace(go.Scatter(x=ncov_uk['ObservationDate'], y=ncov_uk['Confirmed'], name='UK'))
fig1.add_trace(go.Scatter(x=ncov_germany['ObservationDate'], y=ncov_germany['Confirmed'], name='Germany'))
fig1.add_trace(go.Scatter(x=ncov_iran['ObservationDate'], y=ncov_iran['Confirmed'], name='Iran'))
fig1.add_trace(go.Scatter(x=ncov_france['ObservationDate'], y=ncov_france['Confirmed'], name='France'))

fig1.layout.update(title_text='COVID-19 Growth in Rest of the World',xaxis_showgrid=False, yaxis_showgrid=False, width=800,
        height=500,font=dict(
        size=12,
        color="white"
    ))
fig1.layout.plot_bgcolor = 'Black'
fig1.layout.paper_bgcolor = 'Black'
fig1.show()

#### - We see that none of the countries have approached the inflection point we discussed earlier in case of China yet.

In [ ]:
ncov_all['CD_Ratio'] = ncov_all['Deaths']/ncov_all['Confirmed']
ncov_all['CR_Ratio'] = ncov_all['Recovered']/ncov_all['Confirmed']
ncov_all = ncov_all.round(2)
ncov_all.head()

### Comparing the Death:Confirmed & Recovered:Confirmed Ratios

In [ ]:
ncov_all_cdr = ncov_all.sort_values(by=['CD_Ratio'], ascending=False).reset_index(drop=True)
ncov_all_crr = ncov_all.sort_values(by=['CR_Ratio'], ascending=False).reset_index(drop=True)
fig = px.bar(ncov_all_cdr, x="Country", y="CD_Ratio", color='CD_Ratio', title='Country-wise Death:Confirmed Cases Ratio')
fig.show()

- 1 out of 10 person dies in Italy which astonishingly high compared to say, China

In [ ]:
fig = px.bar(ncov_all_crr, x="Country", y="CR_Ratio", color='CR_Ratio', title='Country-wise Recovered:Confirmed Cases Ratio')
fig.show()

### Some pressing questions:
#### i) What did China do there is a really large recovery rate? 
#### ii) Why Iran, Italy, Spain and UK saw a very high Confirmed to dead ratio 

In [ ]:
df2_china = df2[df2['country']=='China'].dropna(subset=['age']).reset_index(drop=True)
df2_italy = df2[df2['country']=='Italy'].dropna(subset=['age']).reset_index(drop=True)
df2_china.head(2)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df2_china['age'], name='China'))
fig.add_trace(go.Box(y=df2_italy['age'], name='Italy'))
fig.update_layout(title='Age Comparison between Chinese and Italians')

fig.show()

In [ ]:
def clean(x):
    if x == 'death' or x == 'died' or x == 'Death':
        return 'death'
    elif x == 'discharged' or x=='discharge':
        return 'discharge'
    elif x == 'recovered' or x=='stable':
        return 'recovered'
    else:
        return np.nan
    
def apply_int(x):
    try:
        y = int(x)
        return y
    except:
        return np.nan

    
df1_chinese = pd.DataFrame(df2_china[df2_china['outcome'].apply(clean)=='death']['age'].apply(apply_int)).assign(outcome='death')
df2_chinese = pd.DataFrame(df2_china[df2_china['outcome'].apply(clean)=='discharge']['age'].apply(apply_int)).assign(outcome='discharge')
df3_chinese = pd.DataFrame(df2_china[df2_china['outcome'].apply(clean)=='recovered']['age'].apply(apply_int)).assign(outcome='recovered')

fig = go.Figure()
fig.add_trace(go.Box(y=df1_chinese['age'], name="Deceased Patients"))
fig.add_trace(go.Box(y=df2_chinese['age'], name="Discharged Patients"))
fig.add_trace(go.Box(y=df3_chinese['age'], name="Recovered Patients"))
fig.update_layout(title_text='Chinese COVID-19 Patients Outcome Age-Wise')
fig.show()

## Lets analyse a bit of India's situation right now

In [ ]:
df_india= pd.read_csv('../input/coronavirus-cases-in-india/Covid cases in India.csv')
dbd_India = pd.read_excel('../input/coronavirus-cases-in-india/per_day_cases.xlsx',sheet_name='India')
dbd_tc_india = pd.read_excel('../input/covid19-india-complete-data/COVID19 India Complete Dataset April 2020.xlsx', sheet_name='Daily Cases Time-Series')

In [ ]:
dbd_tc_india.head(2)

In [ ]:
dbd_India.head(2)

In [ ]:
db_india = pd.DataFrame(dbd_tc_india.groupby(['Date'])['Total Confirmed'].sum()).reset_index()

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=db_india['Date'], y=db_india['Total Confirmed']))
fig1.layout.update(title_text='COVID-19 Growth in India',xaxis_showgrid=False, yaxis_showgrid=False, width=800,
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig1.layout.plot_bgcolor = 'Black'
fig1.layout.paper_bgcolor = 'Black'
fig1.show()

### According to the Western countries (Europe & USA), while we note even although the curve is in the direction of progressively increasing, the amount of cases have not risen significantly. As for those countries we see that the amount of cases in the multiples of 1000 have risen exponentially over a period of 2-3 weeks, while in India it is still below 1000 

### Following are the possible reasons:

#### - **Yes. The amount of tests carried out is smaller than in many nations, yet even with this the number of instances having a large number of 1000s like 50k or 60k is very ridiculous. **
#### - **In India, maybe because of climatic conditions or any other reasons that we may miss out on, Community transmission has never been triggered? **
#### - **No transfer to the community is yet happening**
#### - **Were citizens here better immunized? **
#### - **This Man**

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTclwehjdactJoKW_HSjcOufn4Ieci8sUlw12Ctf4PDQpKwVO_i&s)

### State-Wise No. of cases in India

In [ ]:
db_state_india = pd.read_excel('../input/covid19-india-complete-data/COVID19 India Complete Dataset April 2020.xlsx', sheet_name='State-Wise Data')
db_state_india = db_state_india[db_state_india['State']!='Total']
db_state_india.head(2)

In [ ]:
fig = px.bar(db_state_india.sort_values('Confirmed', ascending=False).sort_values('Confirmed', ascending=True),
             x="Confirmed", y="State", 
             title='Total Confirmed Cases', 
             text='Confirmed', 
             orientation='h', 
             width=800, height=800, range_x = [0, max(db_state_india['Confirmed'])])
fig.update_traces(marker_color='#670404', opacity=0.8, textposition='inside')

fig.update_layout(plot_bgcolor='#CDCCA7')
fig.show()

In [ ]:
dbd_testing_india = pd.read_excel('../input/covid19-india-complete-data/COVID19 India Complete Dataset April 2020.xlsx', sheet_name='ICMR Testing Count')
dbd_testing_india['Update Time Stamp'] = pd.to_datetime(dbd_testing_india['Update Time Stamp'], format='%d/%m/%Y %I:%M: %p')
dbd_testing_india.head(2)

In [ ]:
df_hos_bed = dbd_testing_india.rename(columns={'Update Time Stamp':'DateTime', 'Total Individuals Tested':'TotalIndividualsTested', 'Total Positive Cases':\
                                              'TotalPositiveCases'}).copy()
df_hos_bed['DateTime'] = df_hos_bed['DateTime'].dt.date
df_hos_bed.head()

In [ ]:
df_hos_bed['totalnegative'] = df_hos_bed['TotalIndividualsTested'] - df_hos_bed['TotalPositiveCases']

In [ ]:
df_hos_bed_per_day = df_hos_bed.drop_duplicates(subset=['DateTime'], keep='last')
df_hos_bed_per_day['test_results_posratio'] = round(df_hos_bed_per_day['TotalPositiveCases']/df_hos_bed_per_day['TotalIndividualsTested'], 3)
df_hos_bed_per_day.head()

In [ ]:
df_indi = pd.read_csv('../input/covid19-in-india/IndividualDetails.csv')
df_indi.head(2)

In [ ]:
df_indi.dropna(subset=['current_status', 'age'], inplace=True)
df_indi.reset_index(drop=True, inplace=True)

In [ ]:
df_indi['current_status'].unique(), df_indi.shape

In [ ]:
df1_indians = df_indi[df_indi['current_status'] == 'Deceased']
df2_indians = df_indi[df_indi['current_status'] == 'Hospitalized']
df3_indians = df_indi[df_indi['current_status'] == 'Recovered']

fig = go.Figure()
fig.add_trace(go.Box(y=df1_indians['age'], name="Deceased Patients"))
fig.add_trace(go.Box(y=df2_indians['age'], name="Hospitalized Patients"))
fig.add_trace(go.Box(y=df3_indians['age'], name="Recovered Patients"))
fig.update_layout(title_text='Indian COVID-19 Patients Outcome Age-Wise')
fig.show()

### - We see that outcome trends are similar to China w.r.t. Age

In [ ]:
pep_no_trav_his = df_indi[df_indi['notes'].str.contains('Travel') == False]
pep_with_trav_his = df_indi[df_indi['notes'].str.contains('Travel') == True]

In [ ]:
df_indi['id'].nunique(), pep_no_trav_his['id'].nunique()

In [ ]:
colors = ['#B5B200', '#1300B5']
negative = round(pep_no_trav_his['id'].nunique()/df_indi['id'].nunique()*100, 2)
positive = round(pep_with_trav_his['id'].nunique()/df_indi['id'].nunique()*100, 2)
                         
fig = px.pie(pep_no_trav_his, values=[negative, positive], names=['Patients w/o Travel History', 'Patients with Travel History'], \
             title='Patients with and without Travel History')
fig.show()

### What might be the reasons that patients without travel history got infected?

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='black',
        stopwords=stopwords,
        max_words=200,
        max_font_size=40, 
        scale=3,
        random_state=69 # chosen at random by flipping a coin; it was heads
).generate(str(data))

    fig = plt.figure(1, figsize=(15, 15))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

show_wordcloud(pep_no_trav_his['notes'].apply(lambda x: x.replace('travel', '')))

### - **From the report, it appears like the other patients that have been affected are mostly the close with travel history and few other exceptions. **

### COVID19 Test Results in India

In [ ]:
colors = ['#269A06', '#AF0E06']
df_hos_bed_per_day_clean = df_hos_bed_per_day.dropna(subset=['TotalPositiveCases']).reset_index(drop=True)
df_hos_bed_per_day_clean['TotalIndividualsTested'].fillna(df_hos_bed_per_day_clean['Total Samples Tested']-900, inplace=True)
df_hos_bed_per_day_clean['totalnegative'] = df_hos_bed_per_day_clean['TotalIndividualsTested'] - df_hos_bed_per_day_clean['TotalPositiveCases']
negative = round(df_hos_bed_per_day_clean['totalnegative'].sum()/df_hos_bed_per_day_clean['TotalIndividualsTested'].sum()*100, 2)
positive = round(df_hos_bed_per_day_clean['TotalPositiveCases'].sum()/df_hos_bed_per_day_clean['TotalIndividualsTested'].sum()*100, 2)
fig = go.Figure(data=[go.Pie(labels=['People who tested Negative','People who tested Positive'],
                             values= [negative,positive],hole =.5)])
                          

fig.update_traces(title_text='COVID19 Test Results', hoverinfo='label+percent', textinfo='value', textfont_size=15,
                  marker=dict(colors=colors, line=dict(color='#FFFFFF', width=2)))
fig.show()

### COVID19 Test Results in Rest of the World

In [ ]:
%%HTML
<div class="infogram-embed" data-id="02d885f1-2ce0-446c-99da-994236912e3a" data-type="interactive" data-title="Total COVID 19 Tests"></div><script>!function(e,i,n,s){var t="InfogramEmbeds",d=e.getElementsByTagName("script")[0];if(window[t]&&window[t].initialized)window[t].process&&window[t].process();else if(!e.getElementById(n)){var o=e.createElement("script");o.async=1,o.id=n,o.src="https://e.infogram.com/js/dist/embed-loader-min.js",d.parentNode.insertBefore(o,d)}}(document,0,"infogram-async");</script>

In [ ]:
df_hos_bed_per_day = df_hos_bed_per_day.dropna(subset=['TotalPositiveCases']).reset_index(drop=True)
df_hos_bed_per_day['TotalIndividualsTested'].fillna(df_hos_bed_per_day['Total Samples Tested']-900, inplace=True)
df_hos_bed_per_day['test_results_posratio'] = round(df_hos_bed_per_day['TotalPositiveCases']/df_hos_bed_per_day['TotalIndividualsTested'], 3)
df_hos_bed_per_day.head(2)

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=df_hos_bed_per_day['DateTime'], y=df_hos_bed_per_day['test_results_posratio']*100, name='Confirmed Cases', \
                         marker=dict(color='#D32210')))
fig1.layout.update(title_text='COVID-19 Positive Detection per Test Ratio in India w.r.t. Time',xaxis_showgrid=False, width=700, yaxis_title='% of Patients Tested +ve',
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig1.layout.plot_bgcolor = '#097E99'
fig1.layout.paper_bgcolor = '#097E99'
fig1.show()

In [ ]:
dbd_tc_india.rename(columns={'Daily Confirmed':'New Cases'}, inplace=True)
dbd_tc_india.head(2)

In [ ]:
ss = []
for i in dbd_tc_india.index:
    if(i!= min(dbd_tc_india.index)):
        lm = dbd_tc_india.loc[i]['New Cases']/dbd_tc_india.loc[i-1]['New Cases']
    else:
        lm = np.NaN
    ss.append(lm)
        
dbd_tc_india['Growth_Rate'] = ss
dbd_tc_india.head(2)

### Analysing the Growth Factor in India

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=dbd_tc_india.iloc[35:(dbd_tc_india.shape[0]-1)]['Date'], y=dbd_tc_india.iloc[35:(dbd_tc_india.shape[0]-1)]['Growth_Rate'], name='Growth Factor', \
                         marker=dict(color='#008040')))
fig1.layout.update(title_text='COVID-19 Growth Factor in India w.r.t. Time',xaxis_showgrid=False, yaxis_showgrid=False, width=700, yaxis_title='Growth Factor',
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig1.layout.plot_bgcolor = '#4d3900'
fig1.layout.paper_bgcolor = '#4d3900'
fig1.show()

In [ ]:
print('Mean Growth Factor in India = ', round(dbd_tc_india.iloc[35:]['Growth_Rate'].mean(), 2))